In [12]:
import os

if "notebooks" in os.getcwd():
    os.chdir("..")

In [13]:
from tqdm.notebook import tqdm

In [ ]:
from azure_cosmos_db import *

azure_cosmos = AzureCosmos()

azure_cosmos.list_containers()

# Events, Dice & Shotgun Scraper

In [15]:
SCRAPER_NAME = "shotgun_links"
INPUT_FILE_PATH = "data/inputs/Shotgun Analysis - Non Duplicates.csv"
SHEET_NAME = ""

In [ ]:
azure_cosmos.DATABASE_ID = "Scraper"
azure_cosmos.CONTAINER_NAME = SCRAPER_NAME

azure_cosmos.container = azure_cosmos.initialize_cosmosdb()

In [ ]:
SHEET_NAME, extension = os.path.splitext(os.path.basename(INPUT_FILE_PATH))

if extension==".csv":
    df = pd.read_csv(INPUT_FILE_PATH)
elif extension==".xlsx":
    df = pd.read_excel(INPUT_FILE_PATH, sheet_name="Sheet4")

print(f"Total: {df.shape}")
df.drop_duplicates(inplace=True)
print(f"Unique: {df.shape}")

print(SHEET_NAME)
# df = df.sample(5)
df.head()

In [ ]:
urls = df["Non Duplicates Organisers"].to_list()

for url in tqdm(urls, desc=f"Uploading {len(urls)} URLs to {SCRAPER_NAME}"):
        print(url)
        secure_url = url.replace("http://", "https://")
        hash_key = SHEET_NAME + secure_url
        data = {
            "id": hashlib.sha256(hash_key.encode()).hexdigest(),
            "url": secure_url,
            "processed": False,
            "sheet_name": SHEET_NAME
        }
        azure_cosmos.create_conversation(conversation_data=data)